In [ ]:
import numpy as np
import sys
from casadi import *

# Import do_mpc package:
import do_mpc
import math 

In [ ]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

In [ ]:
#state variables
eta = model.set_variable(var_type='_x', var_name='eta', shape=(6,1))
v = model.set_variable(var_type='_x', var_name='v', shape=(6,1))

#Velocity vector
# eta_dot = model.set_variable(var_type='_z', var_name='eta_dot', shape=(6,1))
# v_dot = model.set_variable(var_type='_z', var_name='v_dot', shape=(6,1))

# Input
thrust = model.set_variable(var_type='_u', var_name='thrust', shape=(6,1))

Paarameters

In [ ]:
x_g = 0
y_g = 0
z_g = 0
x_b = 0
y_b = 0
z_b = -0.01

#added mass coefficients
X_ud = 6.356673886738176
Y_vd = 7.120600295756984
Z_wd = 18.686326861534997
K_pd = 0.185765630747592
M_qd = 0.134823349429660
N_rd = 0.221510466644690
#drag 
Xu = {"Linear": 13.7, "NonLinear": 141}
Yv = {"Linear": 0, "NonLinear": 217}
Zw = {"Linear": 33, "NonLinear": 190}
Kp = {"Linear": 0, "NonLinear": 1.192}
Mq = {"Linear": 0.8, "NonLinear": 0.470}
Nr = {"Linear": 0, "NonLinear": 1.5}


In [ ]:
m = 13.5 # Mass of the Robot

I_x, I_y, I_z = 0.26, 0.23, 0.37 # Mass Moments of Inertia

L_h, L_w, L_l = 0.378, 0.575, 0.457 

A_F, A_S, A_T = 0.0877, 0.1131, 0.2049

Volume = 0.0135

Ic = np.array([[0.26, 0, 0],
      [0, 0.23, 0],
      [0, 0, 0.37]])

Ap_F, Ap_S, Ap_T = 0.1727, 0.2174, 0.2628

Mass matrix

In [ ]:
#Rigid-body mass
M_RB = np.array([[m, 0, 0, 0, 0, 0],
              [0, m, 0, 0, 0, 0],
              [0, 0, m, 0, 0, 0],
              [0, 0, 0, I_x, 0, 0 ],
              [0, 0, 0, 0, I_y, 0 ],
              [0, 0, 0, 0, 0, I_z ]])
#added mass
M_A = -1 * np.array([[X_ud, 0, 0, 0, 0, 0],
              [0, Y_vd, 0, 0, 0, 0],
              [0, 0, Z_wd, 0, 0, 0],
              [0, 0, 0, K_pd, 0, 0 ],
              [0, 0, 0, 0, M_qd, 0 ],
              [0, 0, 0, 0, 0, N_rd ]])
M = M_RB + M_A

Coriolis

In [ ]:
#the rigid body Coriolis
# v = [u v w p q r]
C_RB = np.array([
    [0, 0, 0, 0, m*v[2], -m*v[1]],
    [0, 0,0, -m*v[2], 0, m*v[0]],
    [0, 0, 0, m*v[1], -m*v[0], 0],
    [0, m*v[2], -m*v[1], 0, -I_z*v[5], -I_y*v[4]],
    [-m*v[2], 0, m*v[0], I_z*v[5], 0, I_x*v[3],],
    [m*v[1], -m*v[0], 0, I_y*v[4], -I_x*v[3], 0],
    ])

# added mass Coriolis
C_A = np.array([
    [0, 0, 0,0, -Z_wd*v[2], Y_vd*v[1]],
    [0, 0, 0, Z_wd*v[2], 0,-X_ud*v[0]], 
    [0, 0, 0, -Y_vd*v[1], X_ud*v[0], 0],
    [0, -Z_wd*v[2], Y_vd*v[1], 0, -N_rd*v[5], M_qd*v[4]],
    [Z_wd*v[2], 0, -X_ud*v[0], N_rd*v[5], 0, -K_pd*v[3]],
    [-Y_vd*v[1], X_ud*v[0], 0, -M_qd*v[4], K_pd*v[3], 0],
    ])
C = C_RB + C_A

Drag

In [ ]:
model_type = "NonLinear"

if model_type == "Linear":

    D = np.array([
        [Xu[model_type], 0, 0, 0, 0, 0],
        [0, Yv[model_type], 0, 0, 0, 0],
        [0, 0, Zw[model_type], 0, 0, 0],
        [0, 0, 0, Kp[model_type], 0, Nr[model_type]],
        [0, 0, 0, 0, Mq[model_type], 0],
        [0, 0, 0, 0, 0, Nr[model_type]]
    ])
elif model_type == "NonLinear":
    
    D = np.array([
        [Xu[model_type] * v[0], 0, 0, 0, 0, 0 ],
        [0, Yv[model_type] * v[1], 0, 0, 0, 0 ],
        [0, 0, Zw[model_type] * v[2], 0, 0, 0 ],
        [0, 0, 0, Kp[model_type] * v[3], 0, 0 ],
        [0, 0, 0, 0, Mq[model_type] * v[4], 0 ],
        [0, 0, 0, 0, 0, Nr[model_type] * v[5] ],        
    ])

G and B

In [ ]:
W = m * 9.81
B = 1000 * Volume * 9.81

# eta = [x y z phi theta epsi]
phi = eta[3]
theta = eta[4]
psi = eta[5]

In [ ]:
G = np.array([
    (W - B) * math.sin(theta),
    -(W - B) * math.cos(theta) * math.sin(phi),
    -(W - B) * math.cos(theta) * math.cos(phi),
    -(y_g*W - y_b*B)*math.cos(theta)*math.cos(phi) + (z_g*W - z_b*B)*math.cos(theta)*math.sin(phi),
    (z_g*W - z_b*B)*math.sin(theta) + (x_g*W - x_b*B)*math.cos(theta)*math.cos(phi),
    -(x_g*W - x_b*B)*math.cos(theta)*math.sin(phi) - (y_g*W - y_b*B)*math.sin(theta)
])

J

In [ ]:
#kinematic equation

J2 = [
    [1, math.sin(phi)*math.tan(theta), math.cos(phi)*math.tan(theta)],
    [0, math.cos(phi), -math.sin(phi)],
    [0, math.sin(phi)/math.cos(theta), math.cos(phi)/math.cos(theta)]
]

J1 = [
    [math.cos(psi)*math.cos(theta), -math.sin(psi)*math.cos(phi) + math.cos(psi)*math.sin(theta)*math.sin(phi), math.sin(psi)*math.sin(phi) + math.cos(psi)*math.cos(phi)*math.sin(theta)],
    [math.sin(psi)*math.cos(theta), math.cos(psi)*math.cos(phi) + math.sin(phi)*math.sin(theta)*math.sin(psi), -math.cos(psi)*math.sin(phi) + math.sin(theta)*math.sin(psi)*math.cos(phi)],
    [-math.sin(theta), math.cos(theta)*math.sin(phi), math.cos(theta)*math.cos(phi)]
]

J = np.array([
    [math.cos(psi)*math.cos(theta), -math.sin(psi)*math.cos(phi) + math.cos(psi)*math.sin(theta)*math.sin(phi), math.sin(psi)*math.sin(phi) + math.cos(psi)*math.cos(phi)*math.sin(theta), 0, 0, 0],
    [math.sin(psi)*math.cos(theta), math.cos(psi)*math.cos(phi) + math.sin(phi)*math.sin(theta)*math.sin(psi), -math.cos(psi)*math.sin(phi) + math.sin(theta)*math.sin(psi)*math.cos(phi), 0, 0, 0],
    [-math.sin(theta), math.cos(theta)*math.sin(phi), math.cos(theta)*math.cos(phi), 0, 0, 0]
    [0, 0, 0, 1, math.sin(phi)*math.tan(theta), math.cos(phi)*math.tan(theta)],
    [0, 0, 0, 0, math.cos(phi), -math.sin(phi)],
    [0, 0, 0, 0, math.sin(phi)/math.cos(theta), math.cos(phi)/math.cos(theta)]

])

In [ ]:
eta_dot = J @ v
v_dot =  -np.linalg.inv(M) @ (C @ v + D @ v + G )

Right Hand Side

In [ ]:
model.set_rhs('eta', eta_dot)
model.set_rhs('v', v_dot)
####